"alg-geom" : Algebraic Geometry (math.AG) 

"dg-ga" : Differential Geometry (math.DG)

"q-alg" : Quantum Algebra (math.QA)

"patt-sol" : Pattern Formation and Solitons (nlin.PS); 

"adap-org" : Adaptation and Self-Organizing Systems (nlin.AO)

"solv-int" : Exactly Solvable and Integrable Systems (nlin.SI) 

"chao-dyn" : Chaotic Dynamics (nlin.CD) 

"comp-gas" : Cellular Automata and Lattice Gases (nlin.CG) 

"chem-ph" : Chemical Physics (physics.chem-ph) 

"mtrl-th" : Materials Science (cond-mat.mtrl-sci)

"cmp-lg" : Computation and Language (cs.CL)

"atom-ph" : Atomic Physics (physics.atom-ph) 

"funct-an" : Functional Analysis (math.FA)

"acc-phys" : Accelerator Physics (physics.acc-ph) 

In [2]:
from bs4 import BeautifulSoup
import urllib
import urllib.request
import io
import re
from math import floor
import random
import pandas as pd

In [3]:
base_url = "http://export.arxiv.org/oai2?verb=ListRecords&"
url = base_url + "set={}&metadataPrefix=arXiv".format("math")

In [4]:
u = urllib.request.urlopen(url, data = None)
f = io.TextIOWrapper(u,encoding='utf-8')
text = f.read()

In [5]:
soup = BeautifulSoup(text, 'xml')

In [6]:
a = [i for i in soup.findAll("record")]

In [7]:
record = a[0]

In [8]:
paper_id = record.find("identifier").text
date = record.find("created").text
title = record.find("title").text
authors = ";".join([author.get_text(" ") for author in record.findAll("author")])
category = record.find("setSpec").text

In [9]:
paper_id

'oai:arXiv.org:0704.0002'

In [50]:
for au in a[0].findAll("author"):
    print()

Streinu Ileana
Theran Louis


In [20]:
a[0]

<record>
<header>
<identifier>oai:arXiv.org:0704.0002</identifier>
<datestamp>2008-12-13</datestamp>
<setSpec>math</setSpec>
</header>
<metadata>
<arXiv xsi:schemaLocation="http://arxiv.org/OAI/arXiv/ http://arxiv.org/OAI/arXiv.xsd">
<id>0704.0002</id><created>2007-03-30</created><updated>2008-12-13</updated><authors><author><keyname>Streinu</keyname><forenames>Ileana</forenames></author><author><keyname>Theran</keyname><forenames>Louis</forenames></author></authors><title>Sparsity-certifying Graph Decompositions</title><categories>math.CO cs.CG</categories><comments>To appear in Graphs and Combinatorics</comments><msc-class>05C85; 05C70; 68R10; 05B35</msc-class><license>http://arxiv.org/licenses/nonexclusive-distrib/1.0/</license><abstract>  We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use
it obtain a characterization of the family of $(k,\ell)$-sparse graphs and
algorithmic solutions to a family of problems concerning tree decompositions of
graphs. Special